#Import Libraries

In [ ]:
from google.cloud import bigquery
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date
import sys
sys.path.insert(1, '/home/jupyter')
from src import config, diagnosis, encounters, patients, filter, query, lab

In [ ]:
client = bigquery.Client()

In [ ]:
include_icd = ['F11.20', 'F11.23', 'F11.921', 'F11.929', 'F11.988', 'F11.94', 'F11.981', 'F11.982', 'F91.3']

In [ ]:
#Get relevant diagnosis code
diagnosis_code = diagnosis.get_diagnosis_code_by_icd(client, include_icd)
diagnosis_code.to_csv(r'datasets/opioid_use_diagnosis.csv')
diagnosis_code.info()

In [ ]:
#Get relevant patients who were diagnosis with diases
patients_opioid = patients.get_patients_with_diagnosis(client, diagnosis_code)
print('# of unique patient diagnoised : ', patients_opioid.PATIENT_CLINIC_NUMBER.nunique())

#Inclusion & Exclusion criteris of comorbities

In [ ]:
include_comorbities = ["depression"]
#Get relevant diagnosis code for comorbidities
diagnosis_code_comorbities = diagnosis.get_diagnosis_code((client, include_comorbities))
diagnosis_code_comorbities.info()

In [ ]:
diagnosis_code_comorbities.to_csv(r'datasets/diagnosis_depression.csv')


In [ ]:
#Get relevant patients who were diagnosed with diases and have comorbidity
opioid_depression_patients = patients.get_patients_with_comorbidity(client, diagnosis_code_comorbities, patients_opioid, "has_depression")
opioid_depression_patients.has_depression.value_counts()

In [ ]:
include_comorbities = ["anxiety"]
#Get relevant diagnosis code for comorbidities
opioid_anxiety_patients = patients.get_patients_with_comorbidity(client, diagnosis_code_comorbities, patients_opioid, "has_anxiety")
opioid_anxiety_patients.has_anxiety.value_counts()

In [ ]:
opioid_patients_without_depression = opioid_depression_patients[(opioid_depression_patients.has_depression == 0)]
opioid_patients_without_depression.info()

In [ ]:
opioid_patients_without_anxiety = opioid_anxiety_patients[(opioid_anxiety_patients.has_anxiety == 0)]
opioid_patients_without_anxiety.info()

#Retreiving clinical encounter for patient cohort

Outpatient Encounters


In [ ]:
include_visits = ["PRC", "IAP", "PM", "FAM", "CIM"]

In [ ]:
%%time
#Get encounters for last 5 years
outpatient_encounters = encounters.get_epic_outpatient_encounters_for_cohort(client, opioid_depression_patients, include_visits)
outpatient_encounters.info()

In [1]:
outpatient_encounters.head()

NameError: name 'outpatient_encounters' is not defined

In [ ]:
outpatient_encounters.PATIENT_DK.nunique()

In [ ]:
%%time
#Get all impatient encounters
inpatient_encounters = encounters.get_encounters_for_cohort(client, opioid_patients_without_depression)
inpatient_encounters.info()

#

#Lab Results

In [ ]:
lab_results = lab.get_lab_result_by_encounters(client, consult.head(1000), 'APPOINTMENT_ID')